# How does simpy use generators ?

Remember the Process we wrote in the basic Simpy example.

The process uses yield to wait for the end of the visit

In [1]:
from simpy import Environment, Process
class Visitor(Process):  
    def __init__(self, env):
        super().__init__(env, self.visit())

    def visit(self): 
        print("Here I am at %2.1f" % self.env.now) 
        duration = 2
        yield env.timeout(duration)
        print("I must leave at %2.1f" % self.env.now) 

What is this yield keyword ?

yield is the keywork Python uses to create generators.

A generator is an handy way to build an iterator.

Well, what is an iterator ?

## What is an iterator ?

You actually used iterators before.

For instance, you may go through a list using an iterator. 
In the following case, for will ask an iterator from words
and iterate over

In [2]:
fruits = [ "Apple", "Orange", "Banana"]
for word in fruits:
    print(word)

Apple
Orange
Banana


One of the beauties of iterators is that I do not need to know the internals of the collection.

For instance, I may iterate over a set of words with the same implementation.

In [3]:
fruits_set = ( "Apple", "Orange", "Banana" )
for word in fruits_set:
    print(word)

Apple
Orange
Banana


Iterators are often used with numbers.

In [4]:
numbers = [ 1, 2, 3, 4 ]
for number in numbers:
    print(number)

sum_numbers = sum(numbers)
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


Itertools defines a bunch of iterators for common purposes.

https://docs.python.org/3/library/itertools.html

For instance repeat yield an list of the same number.

In [5]:
from itertools import repeat
for number in repeat(10, 3) :
    print(number)

10
10
10


Some iterators are infinite. 

For instance, count generate number starting from start and incremented by step.

Say we want to generate oddf numbers. As the list is infinite for will loop forever.
Instead we will read the iterator one by one using next.

In [6]:
from itertools import count
odd_number = count(1,2)
print(next(odd_number))
print(next(odd_number))
print(next(odd_number))

1
3
5


## How does an iterator work ?

Internally, the for mecanism thqt goes throug the list is similar to the code snippet below. 

The for expression creates an iterator and call next to get each value until there is no more value to read. 
When the all fruits have been drained the list object raise an exception StopIteration. As a result for stops iterating over the list.

In [7]:
import sys
words = [ "Apple", "Orange", "Banana"]
words_iterator = iter(words)
try:
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
    print(next(words_iterator))
except StopIteration as e:
    print("Unexpected error:", sys.exc_info()[0])

Apple
Orange
Banana
Unexpected error: <class 'StopIteration'>


## How to write an iterator ?

What does it requires ?

Say I want to create a Fruit Basket and iterate over.

In [16]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]

basket = FruitBasket()
try:
    for fruit in basket:
        print(fruit)
except:
    print("Error:", sys.exc_info())

Error: (<class 'TypeError'>, TypeError("'FruitBasket' object is not iterable"), <traceback object at 0x106084880>)


It complains about FruitBasket not being iterable.

Hmmm, what does it mean to be iterable ?

Let us refactor this a bit in order to be iterable.

First iterable defines a behavior. It tells that you can loop over the object. There are basically two ways of looping over the pbject
- provide a mean to get the element at a given index and implement __getitem__
- or provide an iteerator and implement __iter__

As we define iterable partly in terms or iterator, what is an iterator ?

An iterator allow to loop over the elements without the need of an index. Instead, it provides a way to get the next element and implement a method __next__. As a consequence, an iterator must maintain the state of the loop by itself.

By the way, methods named with enclosing __ are special. They provide some well known contract. 

For instance:
- __init__ construct an object of the class type
- __repr__ outputs a representation of the pbject as a string

Let us implement FruitBasket so that it is iterable.

For instance, this version is iterable. 
For will be in charge of computing each index values 
up to the point where index is out of bound.

In [18]:
class FruitBasketWithIterable:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __getitem__(self, pos):
        if (pos > len(self.fruits)):
            raise IndexError
        return self.fruits[pos]

basket = FruitBasketWithIterable()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


This one is iterable and rely on the iterator of the internal list.

In [19]:
class FruitBasketWithIterator:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        
    def __iter__(self):
        return iter(self.fruits)

basket = FruitBasketWithIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


What would be a FruitBasket iterator ?

An iterator must provide a __next__ method, keep track of  the state of the loop,  
and is self-iterable (meaning returns self as __iter__ )

In [20]:
class FruitBasketIterator:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        self.state = 0
        self.limit = len(self.fruits)

    def __iter__(self):
        return self

    def __next__(self):
        if self.state >= self.limit:
            raise StopIteration
        value = self.fruits[self.state]
        self.state += 1
        return value
    
basket = FruitBasketIterator()
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


You may also iterate over an infinite list.

For instance, let us implement the odd numbers iterator 

In [22]:
class OddNumbers:
    def __init__(self):
        self.state = -1

    def __iter__(self):
        return self

    def __next__(self):
        self.state += 2
        return self.state 

odds_iter = OddNumbers()
print(next(odds_iter))
print(next(odds_iter))
print(next(odds_iter))

1
3
5


## And now the generator

Generators provides a convenient way to write iterators.

TODO
The main feature of generator is evaluating the elements on demand. When you call a normal function with a return statement the function is terminated whenever it encounters a return statement. In a function with a yield statement the state of the function is “saved” from the last call and can be picked up the next time you call a generator function.

Let us compare the OddNumbers implementation with this one

In [23]:
def odd_numbers_generator():
    n = 1
    while True:
        yield n
        n += 2
        
odds_gen = odd_numbers_generator()
print(next(odds_gen))
print(next(odds_gen))
print(next(odds_gen))

1
3
5


In [ ]:
The generator is an iterator. It may be used the same way. 
For instance, iterate over geneated first n numbers, print and sum.

In [46]:
def numbers_first_n(n):
    i = 1
    while i <= n:
        yield i
        i += 1
for number in numbers_first_n(4):
    print(number)

sum_numbers = sum(numbers_first_n(4))
print(f"sum {sum_numbers}")

1
2
3
4
sum 10


You may also iterate over lists

In [48]:
def words_generator(): 
    words = [ "Apple", "Orange", "Banana"]
    for w in words:
        yield w 
        
for w in words_generator():
    print(w)

Apple
Orange
Banana


Generator may also be used in a class.

Please note that in order to be iterable the class must implement __iter__ and __next__. Also do not put the generator into next as it would recreated each time next is called. The state would be reset and it loops forever.

In [47]:
class FruitBasket:
    def __init__(self):
        self.fruits = [ "Apple", "Orange", "Banana"]
        self.iterator = self.generator()

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.iterator)
      
    def generator(self):
        for w in self.fruits:
            yield w 

basket = FruitBasket()
print(next(basket))
for fruit in basket:
    print(fruit)

Apple
Orange
Banana


In [ ]:
Please note that the self.generator() returns an iterator.

## How does it relates to Simpy events ?

Here is the process we used. 
The action function looks like the generator function in the generator.

In [ ]:
from simpy import Environment, Process
class Visitor(Process):  
    def __init__(self, env):
        super().__init__(env, self.visit())

    def visit(self): 
        print("Here I am at %2.1f" % self.env.now) 
        duration = 2
        yield env.timeout(duration)
        print("I must leave at %2.1f" % self.env.now) 

Here is a Process class that looks more  similar to a process. The process is defined by a list of task.

In [64]:
class Process:
    def __init__(self):
        self.tasks = [ "task1", "task2", "task3"]
        self.iterator = self.generator()
        
    def __iter__(self):
        return self

    def __next__(self):
        return next(self.iterator)
      
    def generator(self):
        for t in self.tasks:
            yield t       
            
process = Process()
for task in process:
    print(task)

task1
task2
task3


In [ ]:
They look similar

Now let us try a more dynamic process. 
Say each time we unpile a task we put another task onto the list. 

In [70]:
class Process:
    def __init__(self):
        self.tasks = [ "task1", "task2", "task3"]
        self.iterator = self.generator()
        
    def __iter__(self):
        return self

    def __next__(self):
        return next(self.iterator)
      
    def generator(self):
        for t in self.tasks:
            yield t 
            self.tasks.append(f"{t}1")

process = Process()
for i in range(0,10):
    print(next(process))

task1
task2
task3
task11
task21
task31
task111
task211
task311
task1111


The generator will let the process define what comes next. 
The environment will use the generator as a standard 
interface to get the next element.

## Multiple yield

In [ ]:
What happens if the generator has multiple yields

In [74]:
def fruits_generator():
    fruits = [ "Apple", "Orange", "Banana"]
    n = 0
    while n < len(fruits):
        yield n
        yield fruits[n]
        n += 1
        
fruits = fruits_generator()
for fruit in fruits:
    print("-")
    print(fruit)

-
0
-
Apple
-
1
-
Orange
-
2
-
Banana


What happens if the generator chain another generator

In [94]:
def numbers_generator():
    n = 0
    while n < 3:
        yield n
        n += 1
        
def fruits_generator():
    fruits = [ "Apple", "Orange", "Banana"]
    gen = numbers_generator()
    n = 0
    while n < 3:        
        yield fruits[next(gen) ]

for fruit in fruits_generator():
    print(fruit)

Apple
Orange
Banana


RuntimeError: generator raised StopIteration